In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
import time
import sklearn
import pandas as pd
from sklearn import preprocessing

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Remove printing error
pd.options.mode.chained_assignment = None

Functions

In [7]:
# Change iv_treatment length to prior days treatment length 

# For only having one positive switch day per stay
def iv_treatment_length_fun(data):
    # Convert to datetime
    data['date'] = pd.to_datetime(data['date'])

    # iv_treatment_length
    cumcount = []
    count = 0
    pos = -1
    flag = 0

    for x in range(len(data)):
        pos += 1
        if pos == len(data) - 1:
            cumcount.append(999) # add count to last one
            break # end
        elif pos == 0:
            cumcount.append(count) # add 0 to first one
            count += 1
        elif data.iloc[x]['stay_id'] == data.iloc[x+1]['stay_id']:
            if data.iloc[x]['iv_flag'] == 1:
                cumcount.append(count)
                count += 1
            elif flag == 1:
                cumcount.append(999)
                count = 0
                flag = 1
            elif data.iloc[x]['stay_id'] != data.iloc[x-1]['stay_id']:
                cumcount.append(999)
                count = 0
            else:
                cumcount.append(count)
                count = 0
                flag = 1
        else:
            if data.iloc[x]['iv_flag'] == 1:
                cumcount.append(count)
                count = 0
            elif flag == 1:
                cumcount.append(999)
                count = 0
                flag = 0
            else:
                cumcount.append(count)
                count = 0
                flag = 0

    #print(cumcount)
    print(len(cumcount))

    data['iv_treatment_length'] = cumcount
    
    return data

In [3]:
# Final label
def po_label_fun(data):
    data.reset_index(inplace=True, drop=True)

    data['po_flag'] = 0

    pos = -1
    for x in range(len(data)):
        pos += 1
        if data.iloc[x]['iv_flag'] == 1:
            data.loc[x,'po_flag'] = 0
        elif data.iloc[x]['iv_flag'] != data.iloc[x]['iv_flag']: # check if NaN 
            if data.iloc[x]['first_po_flag'] == 1:
                data.loc[x,'po_flag'] = 1
            elif data.iloc[x]['first_po_flag'] != data.iloc[x]['first_po_flag']: # check if NaN 
                print('???')
                data.loc[x,'po_flag'] = 1
    
    return data

In [4]:
# Function to remove outliers based using Q(1/3) -+ 2 * IQR
def remove_data_outliers(df, coeff=6, tol=0):
    """Utility function to remove data outliers from the data set using Z-Score method.
    Args:
        df (DataFrame): Input transformed data set.
        coeff (float, optional): Scaling coefficient for outlier removal strategy. Defaults to 6.
        tol (int, optional): Feature tolerance on how many features with NaN values may exist. Defaults to 0.
    Returns:
        Data set (DataFrame): Output data set without outlier values.
        Outliers per variable (Series): Number of outliers remaining per variable.
    """

    # Calculate IQR = Q3 - Q1
    q1, q3 = df.quantile(.25), df.quantile(.75)
    IQR = q3 - q1
    lower_bound = q1 - (coeff * IQR)
    upper_bound = q3 + (coeff * IQR)

    # Set the outliers to unique number
    df[(df < lower_bound) | (df > upper_bound)] = 999999999

    # Store number of outliers per column
    outlier_count = df[df == 999999999].count(axis=0)

    # Set the outliers to NaN
    df = df.replace(999999999, np.NaN)
    #outlier_count2 = df[df == 999999999].count(axis=0)

    # Drop NaN values with option for adjusting threshold
    #df = df.dropna(how='any', thresh=df.shape[1] - tol)

    # Store number of outliers per column
    #outlier_count = df.isna().sum(axis=0)

    # Return complete profiles and outliers per column
    return df, outlier_count#, outlier_count2

In [5]:
# Get difference 
# Difference
def diff_fun(data):
    data2 = data.drop(columns=['date'])

    data_diff = pd.DataFrame()
    data2.set_index('stay_id', inplace=True)
    for stay_id, new_df in data2.groupby(level=0):
        diff_df = new_df.diff()
        diff_df = diff_df.add_suffix('_difference')
        data_diff = pd.concat([data_diff, diff_df], ignore_index=True)

    data2.reset_index(inplace=True)
    data2 = pd.concat([data2, data_diff], axis=1)
    data = pd.concat([data[['date']], data2], axis=1)
    col = data.pop("stay_id")
    data.insert(0, col.name, col)

    return data

In [49]:
def drop_prior_po_fun(data):
    new_data = pd.DataFrame()
    for stay_id, new_df in data.groupby('stay_id'):
        #print(stay_id)
        sub_df = new_df.copy()
        sub_df.reset_index(inplace= True, drop=True)
        for x in range(len(new_df)):
            if new_df.iloc[x]['po_flag'] == 1:
                #print(sub_df.iloc[x])
                sub_df.drop(x, inplace=True)
            else:
                break
        new_data = pd.concat([new_data, sub_df], axis=0)
    new_data.reset_index(inplace= True, drop=True)
    return new_data

In [8]:
def preprocess_fun(data):
    # Split X y
    X = data.drop(columns=['stay_id', 'date', 'iv_flag', 'first_po_flag', 'po_flag', 'iv_treatment_length'])
    y = data['po_flag']
    # Remove outliers
    times = 2
    X2, outlier_count = remove_data_outliers(X, coeff=times)
    # Forward fill based on stay_id and -1 fill missing
    X2 = pd.concat([X2, data['stay_id']], axis=1)
    ffilled_data = X2.groupby('stay_id').transform(lambda p: p.ffill())
    #ffilled_data = ffilled_data.join(combined_data['stay_id'])
    ffilled_features = ffilled_data
    # Filling missing data, normalize features
    min_max_scaler = preprocessing.MinMaxScaler()
    data_scaled = min_max_scaler.fit_transform(ffilled_features)
    processed_data = np.nan_to_num(data_scaled, nan=-1)

    # Add back lables
    feature_names = list(ffilled_features.columns)
    X3 = pd.DataFrame(processed_data, columns=feature_names)
    #X3.drop(columns=['stay_id'], inplace=True)

    # Split into training, validation and testing
    model_data = pd.concat([data[['stay_id', 'date', 'iv_flag', 'first_po_flag', 'po_flag', 'iv_treatment_length']], X3], axis=1)

    return model_data

mimic

In [ ]:
# Import
mimic_data = pd.read_csv(r"catch_22_data.csv")
# Filter for columns in eicu
z = 'stay_id|date|temperature|o2 saturation pulseoxymetry|heart rate|respiratory rate|blood pressure systolic|blood pressure diastolic|blood pressure mean|gcs - eye opening|gcs - motor response|gcs - verbal response'
mimic_data = mimic_data[mimic_data.columns[mimic_data.columns.str.contains(z)]]
# Get difference for each day
mimic_data = diff_fun(mimic_data)
# Convert to datetime 
mimic_data['date'] =  pd.to_datetime(mimic_data['date'])
# Import dates2 data
dates2 = pd.read_csv(r"iv_switch_stayid_dates.csv")
dates2['date'] =  pd.to_datetime(dates2['date'])
# Combine
mimic_data = pd.merge(dates2, mimic_data, on=['stay_id', 'date'], how='inner').sort_values(by=['stay_id', 'date'])
# Change label 
mimic_data = po_label_fun(mimic_data)
# Change treamtnet length
mimic_data = iv_treatment_length_fun(mimic_data)
# Preprocess
mimic_data = preprocess_fun(mimic_data)
# Drop those rows with po before iv
mimic_data = drop_prior_po_fun(mimic_data)
mimic_data

In [53]:
# Save
mimic_data.to_csv('mimic_data.csv', index=False)

eicu

In [ ]:
# Import
eicu_data = pd.read_csv(r"eicu_catch_22_data.csv")
# Get difference for each day
eicu_data = diff_fun(eicu_data)
# Convert to datetime 
eicu_data['date'] =  pd.to_datetime(eicu_data['date'])
# Import dates2 data
eicu_dates = pd.read_csv(r"eicu_iv_switch_stayid_dates.csv")
eicu_dates['date'] =  pd.to_datetime(eicu_dates['date'])
# Combine
eicu_data = pd.merge(eicu_dates, eicu_data, on=['stay_id', 'date'], how='inner').sort_values(by=['stay_id', 'date'])
# Change label 
eicu_data = po_label_fun(eicu_data)
# Change treamtnet length
eicu_data = iv_treatment_length_fun(eicu_data)
# Preprocess
eicu_data = preprocess_fun(eicu_data)
# Drop those rows with po before iv
print(len(eicu_data))
mimic_data = drop_prior_po_fun(mimic_data)
eicu_data

In [ ]:
# Change iv_treatment length to prior days treatment length 

# For only having one positive switch day per stay
def iv_treatment_length_fun(data):
    # Convert to datetime
    data['date'] = pd.to_datetime(data['date'])

    # iv_treatment_length
    cumcount = []
    count = 0
    pos = -1
    flag = 0

    for x in range(len(data)):
        pos += 1
        if pos == len(data) - 1:
            cumcount.append(0) # add count to last one # Changed to 0 for eicu
            break # end
        elif pos == 0:
            cumcount.append(count) # add 0 to first one
            if data.iloc[x]['stay_id'] == data.iloc[x+1]['stay_id']:
                count += 1
            else:
                count += 0
        elif data.iloc[x]['stay_id'] == data.iloc[x+1]['stay_id']:
            if data.iloc[x]['iv_flag'] == 1:
                cumcount.append(count)
                count += 1
            elif flag == 1:
                cumcount.append(999)
                count = 0
                flag = 1
            elif data.iloc[x]['stay_id'] != data.iloc[x-1]['stay_id']:
                cumcount.append(999)
                count = 0
            else:
                cumcount.append(count)
                count = 0
                flag = 1
        else:
            if data.iloc[x]['iv_flag'] == 1:
                cumcount.append(count)
                count = 0
            elif flag == 1:
                cumcount.append(999)
                count = 0
                flag = 0
            else:
                cumcount.append(count)
                count = 0
                flag = 0

    #print(cumcount)
    print(len(cumcount))

    data['iv_treatment_length'] = cumcount
    
    return data

In [10]:
# Slight change 
# Change iv_treatment_length for second stay - see new function above 
eicu_data = iv_treatment_length_fun(eicu_data)
# Remove these stays 
eicu_data = eicu_data[~((eicu_data['iv_treatment_length'] == 0) & (eicu_data['po_flag'] == 1))]

4947


In [15]:
eicu_data

,stay_id,date,iv_flag,first_po_flag,po_flag,iv_treatment_length,blood pressure diastolic0,blood pressure diastolic1,blood pressure diastolic2,blood pressure diastolic3,...,temperature14_current_stay_difference,temperature15_current_stay_difference,temperature16_current_stay_difference,temperature17_current_stay_difference,temperature18_current_stay_difference,temperature19_current_stay_difference,temperature20_current_stay_difference,temperature21_current_stay_difference,temperature_mean_current_stay_difference,temperature_std_current_stay_difference
0,141196,2022-09-03,1.0,NaN,0,0,0.604437,0.597454,0.090909,-1.000000,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,141392,2022-09-02,1.0,NaN,0,0,0.419841,0.406487,0.181818,0.466667,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,141470,2022-09-02,1.0,NaN,0,0,0.360820,0.373143,0.181818,0.777778,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,142405,2022-09-02,1.0,NaN,0,0,0.566136,0.591088,0.090909,0.700000,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,143068,2022-09-02,1.0,NaN,0,0,0.266429,0.296453,0.272727,0.388889,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4942,3335807,2022-09-02,1.0,NaN,0,0,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4943,3340875,2022-09-04,1.0,NaN,0,0,0.509000,0.506214,0.636364,0.388889,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4944,3341168,2022-09-03,1.0,NaN,0,0,0.502302,0.533596,0.363636,0.304348,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4945,3341168,2022-09-04,1.0,NaN,0,1,0.536940,0.541073,0.181818,0.437500,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [14]:
# Slight change 
# Change last 
eicu_data.iloc[[4946], [5]] = 0

In [8]:
# Save
eicu_data.to_csv('eicu_data.csv', index=False)